<a href="https://colab.research.google.com/github/whkwls2653/Emotion-Recognition/blob/main/wav2vec_local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(allfile_datalist, test_size=0.25, random_state=0)
print(len(dataset_train))
print(len(dataset_test))

In [92]:
!pip install transformers==4.27
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 60.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.11.3
    Uninstalling transformers-4.11.3:
      Successfully uninstalled transformers-4.11.3


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [93]:

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import torch
import ast
from transformers import Wav2Vec2Processor
from torch.nn.utils.rnn import pad_sequence

@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor.from_pretrained("/content/gdrive/MyDrive/감정인식_대회/wave2vec2", pad_token_id=49)
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def mapping(self,data):
        with self.processor.as_target_processor():
            ret = self.processor("".join([i if i!='\x1b' else '|' for i in ast.literal_eval(data)])).input_ids
            ret_torch = torch.tensor([int(0 if value is None else value) for value in ret])
        return ret_torch

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature['input_values']} for feature in features]
        # e.g. feature['label'] = "ㅇㅏㄴㄴㅕㅇㅎㅏㅅㅔㅇㅛ"
        label_features = [{"input_ids": self.mapping(feature["label"])} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        batch["labels"] = labels

        return batch


# 자소 단위로 나누어지지 않은 경우 사용 -> 한번 하면 저장해놓기
class DataProc:
    def __init__(self, model_name="/content/gdrive/MyDrive/감정인식_대회/wave2vec2"):
        self.processor = Wav2Vec2Processor.from_pretrained(model_name, pad_token_id=49)

    def to_jaso(self, sentence):
        NO_JONGSUNG = ''
        CHOSUNGS = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
        JOONGSUNGS = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
        JONGSUNGS = [NO_JONGSUNG,  'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

        N_CHOSUNGS, N_JOONGSUNGS, N_JONGSUNGS = 19, 21, 28
        FIRST_HANGUL, LAST_HANGUL = 0xAC00, 0xD7A3 #'가', '힣'    
     
        result = []
        for char in sentence:
            if ord(char) < FIRST_HANGUL or ord(char) > LAST_HANGUL: 
                result.append('|')
            else:          
                code = ord(char) - FIRST_HANGUL
                jongsung_index = code % N_JONGSUNGS
                code //= N_JONGSUNGS
                joongsung_index = code % N_JOONGSUNGS
                code //= N_JOONGSUNGS
                chosung_index = code
                result.append(CHOSUNGS[chosung_index])
                result.append(JOONGSUNGS[joongsung_index])
                if jongsung_index!=0:
                    result.append(JONGSUNGS[jongsung_index])
                
        with self.processor.as_target_processor():
            ret = self.processor("".join(result))
        
        return ret.input_ids

    def prepare_dataset(self, df):
        """
        df.cols = ['audio', 'sentence', 'path']
        """
        df['label'] = df['sentence'].apply(self.to_jaso)
        return df

loading feature extractor configuration file /content/gdrive/MyDrive/감정인식_대회/wave2vec2/preprocessor_config.json
Feature extractor Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

Didn't find file /content/gdrive/MyDrive/감정인식_대회/wave2vec2/tokenizer.json. We won't load it.
loading file /content/gdrive/MyDrive/감정인식_대회/wave2vec2/vocab.json
loading file /content/gdrive/MyDrive/감정인식_대회/wave2vec2/tokenizer_config.json
loading file /content/gdrive/MyDrive/감정인식_대회/wave2vec2/added_tokens.json
loading file /content/gdrive/MyDrive/감정인식_대회/wave2vec2/special_tokens_map.json
loading file None
Adding <s> to the vocabulary
Adding </s> to the vocabulary
Adding <pad> to the vocabulary


In [94]:
# !pip install datasets
# import datasets
from datasets import load_metric, load_from_disk, Dataset
# from data_proc import  get_senior_data
# from data_collator import DataCollatorCTCWithPadding
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from transformers import TrainingArguments, Trainer
from tqdm import tqdm
import torch
import ast
import pickle
import numpy as np

# from torch.utils.data import Dataset

class MyDataset(Dataset):

  def __init__(self, data_list):
    self.data_list = data_list
    self.len=len(data_list)
    print("len of dataset is",self.len)

  def __getitem__(self, index):
    return self.data_list[index][0],self.data_list[index][2]

  def __len__(self):
    
    return self.len

class MyDataset_2(Dataset):

  def __init__(self, x, y):
    data_dict = dict()
    for i, elem in enumerate(x):
      data_dict[elem] = y[i]
    self.data_dict = data_dict
    self.len=len(data_dict)
    print("len of dataset is",self.len)
    print(type(data_dict))
    print("items",data_dict.items())
    
  def __getitem__(self, index):
    d =list(self.data_dict.items())[index]
    return d[0], d[1]

  def __len__(self):
    return self.len

# import os
# os.environ['CUDA_LAUNCH_BLOCKING']='1'
# os.environ['CUDA_VISIBLE_DEVICES'] = "2,3"

repo_name = '/content/gdrive/MyDrive/감정인식_대회/wave2vec2'

processor = Wav2Vec2Processor.from_pretrained('/content/gdrive/MyDrive/감정인식_대회/wave2vec2')
wer_metric = load_metric("wer")
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    pred.label_ids[pred.label_ids == -100] = 49 # as fleek model has 2 pad tokens
    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

def get_model(model_name="/content/gdrive/MyDrive/감정인식_대회/wave2vec2"): 
    model = Wav2Vec2ForCTC.from_pretrained(
        model_name, 
        attention_dropout=0.1,
        hidden_dropout=0.1,
        feat_proj_dropout=0.0,
        mask_time_prob=0.05,
        layerdrop=0.1,
        ctc_loss_reduction="mean", 
        pad_token_id=49,
        vocab_size=50,
        ignore_mismatched_sizes=True
    )

    model.freeze_feature_extractor()
    model.gradient_checkpointing_enable()
    return model

def train_model(train, test, model):
    training_args = TrainingArguments(
        output_dir=repo_name,
        group_by_length=True,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        gradient_accumulation_steps=4,
        evaluation_strategy="steps",
        num_train_epochs=30,
        fp16=False,
        save_steps=300,
        eval_steps=300,
        logging_steps=50,
        learning_rate=3e-4,
        warmup_steps=300,
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model='eval_loss',
        dataloader_num_workers=6
        )

    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train,
        eval_dataset=test,
        tokenizer=processor.feature_extractor,
        data_collator=data_collator
    )
    print(f"build trainer on device {training_args.device} with {training_args.n_gpu} gpus")
    trainer.train()

if __name__ == "__main__":
    
    # torch.cuda.empty_cache()
    
    # dataset = load_from_disk('/content/gdrive/MyDrive/감정인식_대회/데이터셋/2019wav_txt_label_zip_toteval.pkl')
    # dataset = dataset.remove_columns(['wav', 'text', 'labels'])

    # train = dataset['train'] #.select([i for i in range(0,5000)])
    # test =  dataset['valid'] #.select([i for i in range(0,5000)])
  wav_txt_label_root='/content/gdrive/MyDrive/감정인식_대회/데이터셋/2019wav_txt_label_final.pkl'
  with open(wav_txt_label_root,'rb') as f:
    allfile_datalist=pickle.load(f)
  print(np.array(allfile_datalist).shape)
  # print(len(allfile_datalist))
  # print(allfile_datalist[0])
  train_wav = np.array(allfile_datalist[:10000])[:,0]
  train_label = np.array(allfile_datalist[:10000])[:,2]
  print(len(train_wav))
  
  test_wav = np.array(allfile_datalist[10000:])[:,0]
  test_label = np.array(allfile_datalist[10000:])[:,2]
  train=MyDataset_2(train_wav,train_label)
  print(len(test_wav))
  test=MyDataset_2(test_wav,test_label)

  

  model = get_model()
  train_model(audio_dataset,audio_dataset, model)

loading feature extractor configuration file /content/gdrive/MyDrive/감정인식_대회/wave2vec2/preprocessor_config.json
Feature extractor Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

Didn't find file /content/gdrive/MyDrive/감정인식_대회/wave2vec2/tokenizer.json. We won't load it.
loading file /content/gdrive/MyDrive/감정인식_대회/wave2vec2/vocab.json
loading file /content/gdrive/MyDrive/감정인식_대회/wave2vec2/tokenizer_config.json
loading file /content/gdrive/MyDrive/감정인식_대회/wave2vec2/added_tokens.json
loading file /content/gdrive/MyDrive/감정인식_대회/wave2vec2/special_tokens_map.json
loading file None
Adding <s> to the vocabulary
Adding </s> to the vocabulary
Adding <pad> to the vocabulary


(19257, 3)
10000
len of dataset is 5926
<class 'dict'>
items dict_items([('/content/gdrive/MyDrive/감정인식_대회/데이터셋/KEMDy19/wav/Session01/Sess01_script01/Sess01_script01_M001.wav', '1'), ('/content/gdrive/MyDrive/감정인식_대회/데이터셋/KEMDy19/wav/Session01/Sess01_script01/Sess01_script01_F001.wav', '0'), ('/content/gdrive/MyDrive/감정인식_대회/데이터셋/KEMDy19/wav/Session01/Sess01_script01/Sess01_script01_M002.wav', '0'), ('/content/gdrive/MyDrive/감정인식_대회/데이터셋/KEMDy19/wav/Session01/Sess01_script01/Sess01_script01_M003.wav', '0'), ('/content/gdrive/MyDrive/감정인식_대회/데이터셋/KEMDy19/wav/Session01/Sess01_script01/Sess01_script01_F002.wav', '0'), ('/content/gdrive/MyDrive/감정인식_대회/데이터셋/KEMDy19/wav/Session01/Sess01_script01/Sess01_script01_M004.wav', '0'), ('/content/gdrive/MyDrive/감정인식_대회/데이터셋/KEMDy19/wav/Session01/Sess01_script01/Sess01_script01_F003.wav', '0'), ('/content/gdrive/MyDrive/감정인식_대회/데이터셋/KEMDy19/wav/Session01/Sess01_script01/Sess01_script01_M005.wav', '0'), ('/cont

loading configuration file /content/gdrive/MyDrive/감정인식_대회/wave2vec2/config.json
Model config Wav2Vec2Config {
  "_name_or_path": "hyyoka/wav2vec2-xlsr-korean-senior",
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer",

len of dataset is 5507
<class 'dict'>
items dict_items([('/content/gdrive/MyDrive/감정인식_대회/데이터셋/KEMDy19/wav/Session16/Sess16_impro02/Sess16_impro02_M001.wav', '4'), ('/content/gdrive/MyDrive/감정인식_대회/데이터셋/KEMDy19/wav/Session16/Sess16_impro02/Sess16_impro02_F002.wav', '4'), ('/content/gdrive/MyDrive/감정인식_대회/데이터셋/KEMDy19/wav/Session16/Sess16_impro02/Sess16_impro02_M002.wav', '4'), ('/content/gdrive/MyDrive/감정인식_대회/데이터셋/KEMDy19/wav/Session16/Sess16_impro02/Sess16_impro02_F003.wav', '4'), ('/content/gdrive/MyDrive/감정인식_대회/데이터셋/KEMDy19/wav/Session16/Sess16_impro02/Sess16_impro02_M003.wav', '3'), ('/content/gdrive/MyDrive/감정인식_대회/데이터셋/KEMDy19/wav/Session16/Sess16_impro02/Sess16_impro02_F004.wav', '4'), ('/content/gdrive/MyDrive/감정인식_대회/데이터셋/KEMDy19/wav/Session16/Sess16_impro02/Sess16_impro02_M004.wav', '3'), ('/content/gdrive/MyDrive/감정인식_대회/데이터셋/KEMDy19/wav/Session16/Sess16_impro02/Sess16_impro02_F005.wav', '3'), ('/content/gdrive/MyDrive/감정인식_대회/데이터

All model checkpoint weights were used when initializing Wav2Vec2ForCTC.

All the weights of Wav2Vec2ForCTC were initialized from the model checkpoint at /content/gdrive/MyDrive/감정인식_대회/wave2vec2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Wav2Vec2ForCTC for predictions without further training.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


build trainer on device cpu with 0 gpus


ValueError: ignored

In [88]:
wavs=[]
txts=[]
labels=[]
for x in allfile_datalist:
  wavs.append(x[0])
  txts.append(x[1])
  labels.append(x[2])
dd=dict()
dd['audio']=wavs
# dd['txt']=txts
# dd['label']=labels
# dd['lengths']=19257
print(dd['audio'][1])

# print(wavs[0])
# print(txts[2])
# print(labels[0])
print(type(dd))

/content/gdrive/MyDrive/감정인식_대회/데이터셋/KEMDy19/wav/Session01/Sess01_script01/Sess01_script01_F001.wav
<class 'dict'>


In [95]:
from datasets import Dataset
audio_dataset=Dataset.from_dict(dd).cast_column('audio',Audio())
    

In [40]:
audio_dataset[0]['audio']

{'path': '/content/gdrive/MyDrive/감정인식 대회/데이터셋/KEMDy19/wav/Session01/Sess01_script01/Sess01_script01_M001.wav',
 'array': array([0.00045776, 0.00015259, 0.00030518, ..., 0.00332642, 0.00311279,
        0.00302124]),
 'sampling_rate': 16000}

a=allfile_datalist[0]
print(a)
for a in allfile_datalist:
  a[0],a[1],a[2]=a[1],a[0],a[2]

print(allfile_datalist[0])
print(allfile_datalist[1])